In [1]:
from ssc import SSC
from ssc_exodus import ExodusNetwork
from ssc_jelly import JellyNetwork
from ssc_slayer import SlayerNetwork

In [2]:
dataset = SSC(
    batch_size=64,
    encoding_dim=100,
    num_workers=4,
    download_dir="./data",
)
dataset.setup()
trainloader = dataset.train_dataloader()
validloader = dataset.val_dataloader()

In [3]:
next(iter(trainloader))[0].shape

torch.Size([64, 250, 100])

In [4]:
from tqdm.auto import tqdm

def cycle_through_trainloader():
    for data, targets in tqdm(trainloader):
        data = data.cuda()
        targets = targets.cuda()

In [6]:
cycle_through_trainloader()

  0%|          | 0/1180 [00:00<?, ?it/s]

In [5]:
dict_args = dict(
    encoding_dim=100,
    n_hidden_layers=2,
    hidden_dim=128,
    tau_mem=100000.0,
    output_dim=35,
    spike_threshold=1.,
    learning_rate=1e-3,
    width_grad=1.,
    scale_grad=.5,
    decoding_func='max_over_time',
    optimizer="sgd",
)
slayer_model = SlayerNetwork(**dict_args, n_time_bins=250).cuda()
init_weights = slayer_model.state_dict()

exodus_model = ExodusNetwork(**dict_args, init_weights=init_weights).cuda()
jelly_model = JellyNetwork(**dict_args, init_weights=init_weights).cuda()

sinabs_model = ExodusNetwork(**dict_args, init_weights=init_weights, backend='sinabs').cuda()


In [6]:
from time import time
import numpy as np

algorithms = ["Jelly", "EXODUS", "SLAYER", "BPTT"]
times = {algo: {"forward": [], "backward": [], "reset": []} for algo in algorithms}

# for algo, model in zip(["Jelly", "BPTT", "EXODUS"], [jelly_model, sinabs_model, exodus_model]):
# for algo, model in zip(["EXODUS"], [exodus_model]):
for algo, model in zip(["Jelly", "EXODUS"], [jelly_model, exodus_model]):
    for i in tqdm(range(3)):
        times_epoch = {"forward": [], "backward": [], "reset": []}
        for data, target in tqdm(trainloader):        
            data = data.cuda()
            target = target.cuda()
            t0 = time()
            model.reset_states()
            t1 = time()
            y_hat = model(data)
            t2 = time()
            y_hat.sum().backward()
            t3 = time()
            times_epoch["reset"].append(t1-t0)
            times_epoch["forward"].append(t2-t1)
            times_epoch["backward"].append(t3-t2)
        for step, t in times_epoch.items():
            times[algo][step].append(np.mean(t))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1180 [00:00<?, ?it/s]

  0%|          | 0/1180 [00:00<?, ?it/s]

  0%|          | 0/1180 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1180 [00:00<?, ?it/s]

  0%|          | 0/1180 [00:00<?, ?it/s]

Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7efb9d108940>
Traceback (most recent call last):
  File "/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    Exception ignored in: self._shutdown_workers()
<function _MultiProcessingDataLoaderIter.__del__ at 0x7efb9d108940>  File "/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1464, in _shutdown_workers

Traceback (most recent call last):
      File "/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
if w.is_alive():
      File "/home/felix/.pyenv/versions/3.8.10/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
self._shutdown_workers()    
assert self._parent_pid == os.getpid(), 'can only test a child process'  File "/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/pyth

  0%|          | 0/1180 [00:00<?, ?it/s]

In [ ]:
for i in tqdm(range(3)):
    t0 = time()
    times_epoch = {"forward": [], "backward": [], "reset": []}
    for data, target in tqdm(trainloader):
        data = data.cuda()
        target = target.cuda()
        t0 = time()
        y_hat = slayer_model(data)
        t1 = time()
        y_hat.sum().backward()
        t2 = time()
        times_epoch["forward"].append(t1-t0)
        times_epoch["backward"].append(t2-t1)
    for step, t in times_epoch.items():
        times["SLAYER"][step].append(np.mean(t))

In [7]:
import numpy as np
for action in times["EXODUS"].keys():
    print(action)
    for model, ts in times.items():
        t = np.array(ts[action])
        print(f"\t{model}: ({np.mean(t):.2e} +- {np.std(t):.2e}) s")
        # np.save(f"timings_{model}.npy", t)

forward
	Jelly: (4.00e-03 +- 2.22e-04) s
	EXODUS: (3.19e-03 +- 4.41e-06) s
	SLAYER: (nan +- nan) s
	BPTT: (nan +- nan) s
backward
	Jelly: (3.12e-03 +- 9.29e-05) s
	EXODUS: (3.53e-03 +- 1.08e-04) s
	SLAYER: (nan +- nan) s
	BPTT: (nan +- nan) s
reset
	Jelly: (1.35e-04 +- 7.64e-07) s
	EXODUS: (2.83e-04 +- 7.65e-07) s
	SLAYER: (nan +- nan) s
	BPTT: (nan +- nan) s


/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/felix/.pyenv/versions/3.8.10/envs/exodus/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scala

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(times).to_csv("times_new.csv")

In [ ]:
# Convert previously saved csv to one line per measurement

times = pd.read_csv("times_new.csv", index_col=0)

In [ ]:
table = [
    pd.DataFrame(
        {"algorithm": algo, "time": t, "step": step}
        for step in times.index
        for algo in times.loc[step].index
        for t in eval((times.loc[step].loc[algo]).replace("nan, ", "").replace("nan", ""))
    )
]
table = pd.concat(table, ignore_index=True)
table.to_csv("times_new_table.csv")

In [ ]:
for step in times.index:
    for algo in times.loc[step].index:
        print(step, algo)
        print(times.loc[step].loc[algo])

In [ ]:
table[(table["algorithm"]=="SLAYER") & (table["step"] == "forward")]

In [ ]:
len(dataset.train_data) / dataset.batch_size